<a href="https://colab.research.google.com/github/allisonlinn/CSUREMM/blob/main/Copy_of_Medium_Try1_(logistic%2C_climate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#install needed packages
!pip install snorkel
!pip install textblob
#import libraries and modules
from google.colab import files
import io
import pandas as pd
#Snorkel
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
#NLP packages
import spacy
from nltk.corpus import stopwords
import string
import nltk
import nltk.tokenize
punc = string.punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
#Supervised learning
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
##Deep learning libraries and APIs
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# upload
uploaded = files.upload()

Saving Sheet1.xlsx - Sheet1.csv to Sheet1.xlsx - Sheet1.csv


In [9]:
# to df
df = pd.read_csv(io.BytesIO(uploaded['Sheet1.xlsx - Sheet1.csv']))

#cleaning
df = df.drop(['publish_date'], axis=1)
df = df.rename(columns = {'headline_text': 'text'})
df['text'] = df['text'].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17095 entries, 0 to 17094
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17095 non-null  object
dtypes: object(1)
memory usage: 133.7+ KB


In [10]:
# define constants

POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

# define function which looks into input words to ascribe labels
def keyword_lookup(x, keywords, label):
  if any(word in x.text.lower() for word in keywords):
    return label
  return ABSTAIN

# define function wwhich assigns a correct label
def make_keyword_lf(keywords, label=POSITIVE):
  return LabelingFunction(
      name = f"keyword_{keywords[0]}",
      f=keyword_lookup,
      resources=dict(keywords=keywords,label=label)
  )

  #these two lists can be further extended
"""positive news might contain the following words' """
keyword_positive = make_keyword_lf(keywords=['curbs', 'solar', 'renewable', 'tax', 'regulation', 'electric', 'green', 'agreement', 'trees', 'decarbonize', 'pledge', 'clean', 'alliance', 'bipartisan', 'reduce', 'adopt', 'vow', 'announce', 'positive', 'happy', 'bright', 'confident', 'encouraged', 'negotiations', 'promote', 'talks' ])

"""negative news might contain the following words"""
keyword_negative = make_keyword_lf(keywords=['withdraw','abandon', 'coal', 'fossil fuel','rescind', 'oil', 'reject', 'emissions', 'methane', 'pollution', 'gas', 'dismisses', 'attacks', 'defeat', 'damage', 'natural gas', 'dead', 'fear', 'foul', 'fails', 'hostile', 'cuts', 'quash', 'victims',  'derail', 'deny', 'harm', 'against', 'disagreement', 'denier', 'weaken', 'bad', 'alarming', 'costs', 'worries', 'dismiss', 'pain', 'poison', 'unfair', 'unhealthy'
                                              ], label=NEGATIVE)

In [11]:
# set up a preprocessor for polarity and subjectivity using textlob
@preprocessor(memoize=True)
def textblob_sentiment(x):
  scores = TextBlob(x.text)
  x.polarity = scores.sentiment.polarity
  x.subjectivity = scores.sentiment.subjectivity
  return x

#find polarity
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
  return POSITIVE if x.polarity > 0.6 else ABSTAIN

#find subjectivity
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
  return POSITIVE if x.subjectivity >= 0.5 else ABSTAIN

In [12]:
#combine labeling functions
lfs = [keyword_positive, keyword_negative, textblob_polarity, textblob_subjectivity ]

#apply the lfs on the df
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df)

#apply the label model
label_model = LabelModel(cardinality=2, verbose = True)
#fit on the data
label_model.fit(L_snorkel)
#predict and create the labels
df["label"] = label_model.predict(L=L_snorkel)

100%|██████████| 100/100 [00:00<00:00, 421.20epoch/s]


In [ ]:
#Logistic Regression: binary classifier, estimates probability of an instance belonging to a class, makes according predictions

#text-preprocessing: tokenizing, lemmatizing, removal of stop words, removal of punctuation

In [13]:
#make a copy of the dataframe
data = df.copy()
#define a function which handles the text preprocessing
def preparation_text_data(data):
    """
    This pipeline prepares the text data, conducting the following steps:
    1) Tokenization
    2) Lemmatization
    4) Removal of stopwords
    5) Removal of punctuation
    """
    # initialize spacy object
    nlp = spacy.load('en_core_web_sm')
    # select raw text
    raw_text = data.text.values.tolist()
    # tokenize
    tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]
    #define the punctuations and stop words
    punc = string.punctuation
    stop_words = set(stopwords.words('english'))
    #lemmatize, remove stopwords and punctuationd
    corpus = []
    for doc in tqdm(tokenized_text):
        corpus.append([word.lemma_ for word in doc[0] if (word.lemma_ not in stop_words and word.lemma_ not in punc)])
    # add prepared data to df
    data["text"] = corpus
    return data

#apply the data preprocessing function
data = preparation_text_data(data)


<ipython-input-13-0bc5bd3568a1>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]


  0%|          | 0/17095 [00:00<?, ?it/s]

<ipython-input-13-0bc5bd3568a1>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(tokenized_text):


  0%|          | 0/17095 [00:00<?, ?it/s]

In [14]:
data

,text,label
0,"[new, jersey, embrace, idea, reject, make, uti...",0
1,"[new, jersey, rejoin, regional, emission, prog...",0
2,"[climate, new, voice, trenton]",-1
3,"[climate, gov, murphy, bring, new, voice, new,...",-1
4,"[new, jersey, governor, put, state, back, envi...",-1
...,...,...
17090,"[buckeye, expect, tough, game, 2nd, time, badger]",0
17091,"[ice, fall, glass, skyscraper, new, york, today]",-1
17092,"[cook, comfort]",-1
17093,"[time, rethink, outsider, art]",-1


In [15]:
def text_representation(data):
  tfidf_vect = TfidfVectorizer()
  data['text'] = data['text'].apply(lambda text: " ".join(set(text)))
  X_tfidf = tfidf_vect.fit_transform(data['text'])
  print(X_tfidf.shape)
  print(tfidf_vect.get_feature_names_out())
  X_tfidf = pd.DataFrame(X_tfidf.toarray())
  return X_tfidf
#apply the TFIDV function
X_tfidf = text_representation(data)

(17095, 11074)
['000' '01' '01st' ... 'zuma' 'zuzana' 'époque']


In [16]:
X= X_tfidf
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#fit Log Regression Model
clf= LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          -1       0.81      0.98      0.89      3672
           0       0.84      0.52      0.64       491
           1       0.88      0.55      0.68      1479

    accuracy                           0.83      5642
   macro avg       0.85      0.68      0.74      5642
weighted avg       0.83      0.83      0.81      5642



In [20]:
new_data = ["new carbon tax expected to fail"]
tf = TfidfVectorizer()
tfdf = tf.fit_transform(data['text'])
vect = pd.DataFrame(tf.transform(new_data).toarray())
new_data = pd.DataFrame(vect)
logistic_prediction = clf.predict(new_data)
print(logistic_prediction)

[1]
